In [ ]:
import pickle

In [ ]:
import torch

with open("../feed_dict.json","rb") as file:
    feed_dict = pickle.load(file)

In [ ]:
feed_dict_q = feed_dict
feed_dict_k = feed_dict

In [ ]:
import torch

feed_dict_q["view"] = torch.rand((feed_dict["image"].size(0),1,7))
feed_dict_k["view"] = torch.rand((feed_dict["image"].size(0),1,7))

In [ ]:
import sys
sys.path.append("../")

In [ ]:
# default_exp model.encoder

In [ ]:
#export

from core.scene_graph.scene_graph import SceneGraph
from torchvision.models import resnet34
import torch.nn as nn
import torch

In [ ]:
#export

class Encoder(nn.Module):
    def __init__(self, dim = 256):
        super().__init__()
        
        self.dim = dim
        self.resnet = resnet34(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(self.resnet.children())[:-3])
        
        self.scene_graph = SceneGraph(feature_dim=self.dim, 
                                 output_dims=[self.dim,self.dim],
                                 downsample_rate=16)
        
        
    def forward(self,feed_dict):
        
        num_batch = feed_dict["images"].shape[0]
        num_total_nodes = feed_dict["objects_length"].sum().item()
        
        image_features = self.feature_extractor(feed_dict["images"])
        outputs = self.scene_graph(image_features, feed_dict["objects"], feed_dict["objects_length"])
        
        node_features = output[0][0]
        for num in range(1,num_batch):
            node_features = torch.cat([node_features, output[num][0]], dim =0)
        
        # To be implemented
        spatial_features = None
        
        return outputs, node_features, spatial_features

In [ ]:
encoder = Encoder()

In [ ]:
output, node_features, spatial_features = encoder(feed_dict)